In [362]:
import nltk, re, pprint
from nltk import word_tokenize
from urllib import request
url = "http://www.imsdb.com/scripts/Ali.html"
MOVIE_NAME = url.split('/')[-1].split('.html')[0]
html = request.urlopen(url).read().decode('utf8',errors='ignore')
if html.count('<pre>') != 1:
    print("WARNING: #of <pre>'s",html.count('<pre>'))
html = html.split('<pre>')[-1].split('pre>')[0] + 'pre>'
print('Movie:',MOVIE_NAME)
html[:500]

Movie: Ali


'           \n<b>          \n</b><b>          \n</b><b>          \n</b><b>                                  ALI\n</b>\n<b>          \n</b><b>          \n</b>                               Written by\n<b>          \n</b>                Stephen J Rivele, Christopher Wilkinson, \n<b> \n</b>                         Eric Roth & Michael Mann\n\n<b>          \n</b><b>          \n</b>                                Story by\n<b>          \n</b>                         Gregory Allen Howard\n\n\n<b>          \n</b><b>          '

In [363]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'html.parser')
raw = soup.get_text()
#print(html)
print(raw)
raw.replace('\r\n','\n')
lines = raw.split('\n')






                                  ALI



                               Written by

                Stephen J Rivele, Christopher Wilkinson, 

                         Eric Roth & Michael Mann



                                Story by

                         Gregory Allen Howard



                                                        FADE IN:

          EXT. MIAMI STREET - MOVEMENT - NIGHT (1964)

          in the dark. Coming toward us. Up and down in sync to an
          INSTRUMENTAL LEAD-IN from somewhere. A slip of light. A
          glimpse of somebody in shadow under a sweatshirt hood,
          staring at us, in and out of the dark as...

          INT. THE STAGE, HAMPTON HOUSE CLUB - EMPTY FRAME - NIGHT

          A man walks into the shot, grabs a microphone, slips out of
          his jacket and looks at us. He wants to tell us something.
          He's in a lavender light. This is SAM COOKE. What he calls
          out...a throaty mixture of gospel, soul and sex..

In [364]:
def is_char(ch):
    if ch==' ' or ch == '\n' or ch == '\r':
        return False
    return True
def is_all_caps(line):
    try:
        if line == "":
            return False
        for c in line:
            if c.islower():
                return False
        return True
    except:
        pass
        #print(line)
def is_speech(i):
    if i == 0 or lines[i].strip() == "": 
        return False
    #if is_char(lines[i][0]):
    #    return False
    if(is_all_caps(lines[i-1].strip()) and lines[i].strip()[0].islower()):
        return True
    if is_scene_changer(lines[i]):
        return False
    #if line.strip()[0].islower():
    #    return False
    return is_speech(i-1)
    
def remove_directions():
    Min = 9999
    for i in range(len(lines)):
        if len(lines[i]) > 0:
            cnt = 0
            while cnt < len(lines[i]) and lines[i][cnt] == ' ':
                cnt += 1
            Min = min(cnt,Min)
    print('Min',Min)
    for i in range(len(lines)):
        if len(lines[i]) > 0:
            lines[i] = lines[i][Min:]
            
    direction_list = ['ROOFTOP',"OMITTED",'Omitted','(ECU)',"Shooting Script","CONTINUED",'BACK TO','CLOSE ON','DISSOLVE TO',
                      'ONSCREEN','FADE','TO CAMERA','CLOSEUP','SUPERIMPOSE','9/27/13','Revised','REVISED','Revision','PAGES',
                     'SCREENPLAY','OMIT','SHOOTING SCRIPT','"IN DARKNESS" SCRIPT',' POV',')B(']
    
        
    for i,line in enumerate(lines):
        if len(line) > 0 and is_char(line[0]) and not is_speech(i) and not is_name(i):
            #lines[i] = ""
            pass
        else:
            for word in direction_list:
                if word in lines[i]:
                    lines[i] = ""
    print('Removing these directions:')
    INDENTATION_FOR_DIRECTION = 0
    for i in range(len(lines)):
        if lines[i] == '' or is_scene_changer(lines[i]):
            continue
        for j in range(INDENTATION_FOR_DIRECTION):
            if is_char(lines[i][j]):
                print(lines[i])
                lines[i] =  ''
                break
    #1 if its direction,-1 otherwise            
    global direction_ar
    
    direction_ar = [0]*len(lines)
    print('Unconnected lines,possible directions:')
    for i in range(len(lines)):
        if check_prev_line(i):
            pass
    Min = 9999
    cnt_ar = [0]*100
    for i in range(len(lines)):
        if direction_ar[i] == 1 and len(lines[i])>0:
            cnt = 0
            while lines[i][cnt] == ' ':
                cnt += 1
            cnt_ar[cnt] += 1
    
    for i in range(25):
        print(i,cnt_ar[i])
        
    for i in range(len(lines)):
        if direction_ar[i] == 1:
            print(lines[i])
            lines[i] = ''
            
def check_prev_line(i):
    global direction_ar
    if i==0 or lines[i] == '' :
        #direction_ar[i] = -1
        return False
    #print(i)
    #print(i,direction_ar[i])
    if direction_ar[i] != 0:
        return direction_ar[i] == 1
    if  is_scene_changer(lines[i]) or is_name(lines[i]) or lines[i].strip()[0] == '(':
        direction_ar[i] = -1
        return False
    if lines[i-1] == '':
        direction_ar[i] = 1
        return True
    
    val = check_prev_line(i-1)
    if val:
        direction_ar[i] = 1
    return val
        
    
def misintented_names():
    for i in range(len(lines)):
        if len(lines[i]) > 0 and is_all_caps(lines[i]) and lines[i][0].isupper():
            if not is_scene_changer(lines[i]):
                print(lines[i])
                #print(lines[i+1])
                lines[i] = ""
def is_scene_changer(line):#deteck scene changer lines
    sc_list = ["EXT.","INT.","EXT ","INT ","DECK ","ROOM","ELEVATOR","ON THE ","SLAM TO","INT/EXT","I/E.","TITLE",'TRANSITION TO',
               'LATER','NIGHT','THAT','APARTMENT','CUT TO','INTERCUT','SCENE','SFX OVER','VFX:']
    for sc in sc_list:
        if(sc in line):
            return True
    
    return False
def is_name(name):
    if not is_all_caps(name):
        return False
    if is_scene_changer(name):
        return False
    if len(name)<2:
        return False
    if name[-1] == '.':
        return False
    return True
with open(MOVIE_NAME+ "_original.txt","w",encoding='utf-8') as f:
    for i,line in enumerate(lines):
        f.write(lines[i]+'\n')
for i,line in enumerate(lines):
    lines[i] = lines[i].replace("’","'")
    lines[i] = lines[i].replace('(MORE)','')#replace all (MORE)
    lines[i] = lines[i].replace('EXT:','EXT.') # default is 'EXT.', not 'EXT:'
    lines[i] = lines[i].replace("cont'd","CONT'D")
    lines[i] = lines[i].replace('(Cont.)',"(CONT'D)")

print('-Those will be removed---------')
misintented_names()
print('-------------------------------')
direction_ar = [0]
remove_directions()


-Those will be removed---------
-------------------------------
Min 10
Removing these directions:
Unconnected lines,possible directions:
0 1872
1 0
2 0
3 0
4 0
5 0
6 1
7 0
8 0
9 25
10 0
11 2
12 0
13 0
14 0
15 2
16 0
17 0
18 0
19 21
20 0
21 3
22 2
23 0
24 0
                     Written by
      Stephen J Rivele, Christopher Wilkinson, 
               Eric Roth & Michael Mann
                      Story by
               Gregory Allen Howard
in the dark. Coming toward us. Up and down in sync to an
INSTRUMENTAL LEAD-IN from somewhere. A slip of light. A
glimpse of somebody in shadow under a sweatshirt hood,
staring at us, in and out of the dark as...
A man walks into the shot, grabs a microphone, slips out of
his jacket and looks at us. He wants to tell us something.
He's in a lavender light. This is SAM COOKE. What he calls
out...a throaty mixture of gospel, soul and sex...is "Let me
hear it!" And WOMEN SHRIEK. He says, "Yeah!" They answer,
shrieking, "Oh, yeah!"...
up and down, running 

In [365]:
print("you shouldn't see anything below here:")
for elem in soup.find_all('b'):
    line = str(elem).split('<b>')[1]
    line = line.split('</b>')[0]
    if is_scene_changer(line):
        continue
    if not is_all_caps(elem):
        print(line)

you shouldn't see anything below here:


In [366]:
for i,line in enumerate(lines):
    print(line)





                        ALI



















EXT. MIAMI STREET - MOVEMENT - NIGHT (1964)






INT. THE STAGE, HAMPTON HOUSE CLUB - EMPTY FRAME - NIGHT








EXT. MIAMI STREET - HOODED MAN'S FACE - NIGHT








INT. THE STAGE, HAMPTON HOUSE CLUB (MIAMI) - SAM COOKE




EXT. MIAMI STREET - CASSIUS - NIGHT

now runs diagonally across NW 7th INTERCUT with Cooke
shouting, "Yeah!"
EXT. MACARTHUR CAUSEWAY - CASSIUS - NIGHT

                     SAM COOKE'S AUDIENCE (O.S.)
         Oh, yeah!












WIDE FROM THE FRONT:      CASSIUS + A METRO-DADE POLICE CAR









INT. A BOARD ROOM - GORDON DAVIDSON - DAY









                     LSG BOARD MEMBER
               (reading)
           "...the successor trustee shall be
           fully authorized to pay or disperse
           such sums from the income or
           principal as may be required."
               (beat)
           Do you understand so far, Mr. Clay?


















                     LSG BOARD MEMBER
      

In [367]:


#remove tabs,(MORE),pagenumbers from text 

print("Modifing the file...")
for i,line in enumerate(lines):
    lines[i] = lines[i].strip()
    lines[i] = lines[i].replace("’","'")
    lines[i] = lines[i].replace('(MORE)','')#replace all (MORE)
    lines[i] = lines[i].replace('EXT:','EXT.') # default is 'EXT.', not 'EXT:'
    lines[i] = lines[i].replace("cont'd","CONT'D")
    lines[i] = lines[i].replace('(Cont.)',"(CONT'D)")
    if(lines[i]==''):
        continue
    lines[i] = lines[i].replace('\t',' ')#replace all tab characters with space

    if("(CONT'D)" in lines[i] and lines[i][-8:] != "(CONT'D)"):
        #print(lines[i])
        cont_splitted = lines[i].split("(CONT'D)")
        lines[i] = cont_splitted[0]
        lines.insert(i+1,"")
        lines.insert(i+2,cont_splitted[1][1:])
        #fix?


Modifing the file...


In [368]:
to_be_printed = []
discovery = 0
print('# lines:',len(lines))
for i,line in enumerate(lines):
    if(line.count("(") == 1):
        #print(line)    
        if(is_all_caps(line)):
            continue
        if(line[0]=='(' and line[-1]==')' and line[1:10] == 'copyright'):
            continue
        if("CONT'D" in line or "O.S." in line or "V.O"  in line or "MORE" in line):
            continue
        if is_scene_changer(line):
            continue
        #print(line)    
        pre_emotion_empty_count = 0
        pre_step = 0
        pre_name_count = 0
        result = []
        while(pre_name_count < 1 or len(lines[i - pre_step]) < 2 or  not  is_all_caps(lines[i-pre_step])):
            if is_all_caps(lines[i-pre_step]) and len(lines[i-pre_step]) > 0:
                pre_name_count += 1
            pre_step += 1
        if(is_scene_changer(lines[i-pre_step])):
            result.append("")
            result.append("")
            result.append("")
            pre_step-=1
            while(lines[i-pre_step] == ""):
                pre_step-=1

        post_emotion_empty_count = 0
        post_step = 0
        post_name_count = 0

        while i + post_step+1 < len(lines) and (post_name_count < 2 or len(lines[i + post_step]) < 2 or  not  is_all_caps(lines[i+post_step]) ):
            if len(lines) <= i+post_step or is_scene_changer(lines[i+post_step]) :
                break
            if is_all_caps(lines[i+post_step]) and len(lines[i+post_step]) > 0:
                post_name_count += 1
            post_step += 1
        while is_scene_changer(lines[i+post_step]):
            post_step -=1

        #print(lines[i-pre_step:i+post_step])
        #print(pre_step,post_step)
        printed_lines = lines[i-pre_step:i+post_step]

        for line_in_range in lines[i-pre_step:i+post_step]:
            if(line_in_range!="" and not line_in_range[:-1].isnumeric()):
                if(line_in_range!="(O.S.)"):
                    if(len(result)%3 == 1 and result[-1][0] != '(' and line_in_range[0]!='('):
                        result.append("")
                    result.append(line_in_range)
        #print("=".join(result))
        name_utterance = []
        tmp_utterance = "%%%"
        for x in result:
            x = str(x)
            if(is_name(x)):
                if(tmp_utterance != "%%%"):
                    name_utterance.append(tmp_utterance)
                tmp_utterance = ""
                name_utterance.append(x.strip())
            else:
                if(x!=""):
                    tmp_utterance +=" "
                tmp_utterance +=""+ x.strip()

        if(tmp_utterance != ""):
            name_utterance.append(tmp_utterance)
        #print("=".join(name_utterance))
        #out.write("\t".join(name_utterance)+"\n")
        if(len(name_utterance) <= 2 or '%%%' in name_utterance[0]):
            continue
        #print(name_utterance)
        if name_utterance[0].split()[0] == name_utterance[2].split()[0]:
            #print("NOT a D:",name_utterance)
            continue



        while(len(name_utterance)<8):
            name_utterance.append("")
        to_be_printed.append(name_utterance + [MOVIE_NAME])
        #print("+".join(result))
        #out.write("\t".join(result)+"\n")
        #print(discovery,i)
        discovery+=1
        #if(discovery>50):
         #   break
    elif(line.count("(")>2):
        print("Line {0} has more than one (",i)
        
with open(MOVIE_NAME + '_out.txt','w') as out:
    for i,l in enumerate(to_be_printed):
        if(i == len(to_be_printed)-1 or not (l[0]==to_be_printed[i+1][0] and l[1]==to_be_printed[i+1][1])):
            out.write("\t".join(l)+"\n")
out.close()
print('DONE!')

# lines: 7973
DONE!
